In [2]:
!pip install tensorflow
!pip install seaborn

In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import time
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# ==============================================================================
# 1. SETUP: DATA LOADING AND PREPROCESSING 
# ==============================================================================

# --- Data Loading and Standardization ---
try:
    # 1. Load data, assuming 'house_prices_dataset.csv' is available
    df = pd.read_csv('house_prices_dataset.csv')
    
    # Logic to identify features and targets
    feature_columns = ['area_scaled', 'rooms_scaled', 'location_score_scaled']
    target_column = 'price'
    
    if not all(col in df.columns for col in feature_columns + [target_column]):
        print(f"Error: CSV must contain columns: {feature_columns + [target_column]}")
        exit()
        
    X = df[feature_columns].values 
    y = df[target_column].values.reshape(-1, 1)
    
    # 2. Apply StandardScaler
    scaler = StandardScaler()
    X_norm = scaler.fit_transform(X) 
    print(f"Loaded {X.shape[0]} samples. Features standardized into X_norm.")

except FileNotFoundError:
    print("FATAL ERROR: 'house_prices_dataset.csv' not found. Exiting.")
    exit()

# --- Data Splitting ---
# We use a standard 70/15/15 split.
TEST_SIZE = 0.15 # 15% for final testing
VAL_RATIO = 0.17647 # (0.15 / 0.85) = 17.647% of the remaining 85% for validation
RANDOM_SEED = 42 # Consistent seed for reproducibility

# First split: Separate Test set
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X_norm, y, test_size=TEST_SIZE, random_state=RANDOM_SEED
)

# Second split: Separate Train and Validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=VAL_RATIO, random_state=RANDOM_SEED
)

print(f"Data Split: Train={X_train.shape[0]}, Val={X_val.shape[0]}, Test={X_test.shape[0]}")

# --- Define Hyperparameters  ---
INPUT_SIZE = X_train.shape[1] 

ACTIVATIONS = ['relu', 'leaky_relu', 'elu', 'selu', 'gelu', 'swish'] # >= 6 required
DEPTHS = [1, 2, 3] # >= 3 hidden layers required
WIDTHS = [8, 16, 32] # >= 3 widths required (6 x 3 x 3 = 54 runs minimum)

LEARNING_RATE = 0.001
EPOCHS = 150 # Max epochs before early stopping
BATCH_SIZE = 32
PATIENCE = 15 # Early stopping patience based on val_loss

results_list = [] # List to store results for the final CSV

Loaded 500 samples. Features standardized into X_norm.
Data Split: Train=350, Val=75, Test=75


In [5]:
# ==============================================================================
# 2. MODEL CREATION FUNCTION (Keras FNN Model)
# ==============================================================================

def create_model(depth, width, activation_name):
    """Dynamically creates the FNN model based on parameters."""
    model = keras.Sequential()
    
    # Determine the activation for the dense layers
    if activation_name == 'leaky_relu':
        # [cite_start]LeakyReLU must be implemented as a layer with alpha=0.01 [cite: 1, 17]
        activation_layer = keras.layers.LeakyReLU(alpha=0.01)
        use_layer = True
    else:
        # Standard activations are available as strings
        activation_layer = activation_name 
        use_layer = False

    # Build Hidden Layers
    for _ in range(depth):
        if use_layer:
            # Add Dense layer then the LeakyReLU layer
            model.add(keras.layers.Dense(width))
            model.add(activation_layer)
        else:
            # Use activation string for other standard functions
            model.add(keras.layers.Dense(width, activation=activation_layer))
            
    # Output Layer (1 neuron, linear activation for regression)
    model.add(keras.layers.Dense(1, activation='linear'))
    
    # Compilation
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    # [cite_start]Loss: MSE is the primary metric [cite: 1, 25]
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    
    return model

In [6]:
# ==============================================================================
# 3. PHASE 2: THE BENCHMARK EXECUTION LOOP
# ==============================================================================

# Dictionary to store loss histories for required plots
history_storage = {}
best_val_r2 = -float('inf')
best_overall_models = [] # To store the top 3 models for loss curves

for activation in ACTIVATIONS:
    for depth in DEPTHS:
        for width in WIDTHS:
            
            print(f"Running: Act={activation}, Depth={depth}, Width={width}")
            
            # --- Initialize Logging ---
            config = {
                'activation': activation,
                'depth': depth,
                'width': width
            }
            
            # --- Model Creation and Parameter Count ---
            model = create_model(depth, width, activation)
            # --- FIX: Explicitly build the model before counting parameters ---
            # X_train is a 2D array (samples, features). We need the shape of ONE sample.
            input_shape = (None, X_train.shape[1]) 
            # The (None, ...) handles the batch size, and X_train.shape[1] is the number of features (3)

            model.build(input_shape)
            n_params = model.count_params()
            config['n_params'] = n_params 

            # [cite_start]--- Training with Runtime Tracking and Early Stopping [cite: 1, 24] ---
            start_time = time.time()
            
            # Callback to save best model based on validation loss, restoring weights
            callbacks = [
                keras.callbacks.EarlyStopping(
                    monitor='val_loss', 
                    patience=PATIENCE, 
                    restore_best_weights=True
                )
            ]
            
            history = model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=EPOCHS,
                batch_size=BATCH_SIZE,
                callbacks=callbacks,
                verbose=0
            )
            
            end_time = time.time()
            config['runtime_sec'] = end_time - start_time
            
            # [cite_start]--- Extract Metrics [cite: 1, 25] ---
            
            # 1. Best Validation Loss and Epoch
            best_val_mse = min(history.history['val_loss'])
            best_epoch = np.argmin(history.history['val_loss']) + 1 
            config['val_mse'] = best_val_mse
            config['best_epoch'] = best_epoch
            
            # [cite_start]2. Evaluate on Test Set (MSE, MAE, R2) [cite: 1, 25]
            test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
            y_pred = model.predict(X_test, verbose=0)
            test_r2 = r2_score(y_test, y_pred)
            
            config['test_mse'] = test_loss
            config['test_mae'] = test_mae
            config['test_r2'] = test_r2
            
            # --- Store History for Loss Curves (Deliverable 3b) ---
            history_storage[f'{activation}_{depth}_{width}'] = history.history

            # --- Track Overall Top 3 Models ---
            # Update the list of best models based on test_r2
            model_summary = {
                'name': f'{activation}_D{depth}_W{width}',
                'test_r2': test_r2,
                'history': history.history
            }
            
            # Keep the top 3 models based on R2 (higher is better)
            best_overall_models.append(model_summary)
            best_overall_models.sort(key=lambda x: x['test_r2'], reverse=True)
            best_overall_models = best_overall_models[:3]
            
            # --- Append to Results ---
            results_list.append(config)
            
# [cite_start]--- Save Results to CSV (Deliverable) [cite: 2, 35] ---
results_df = pd.DataFrame(results_list)

# File will be written in the next code execution block

Running: Act=relu, Depth=1, Width=8
Running: Act=relu, Depth=1, Width=16
Running: Act=relu, Depth=1, Width=32
Running: Act=relu, Depth=2, Width=8
Running: Act=relu, Depth=2, Width=16
Running: Act=relu, Depth=2, Width=32
Running: Act=relu, Depth=3, Width=8
Running: Act=relu, Depth=3, Width=16
Running: Act=relu, Depth=3, Width=32
Running: Act=leaky_relu, Depth=1, Width=8


  warnings.warn(


Running: Act=leaky_relu, Depth=1, Width=16


  warnings.warn(


Running: Act=leaky_relu, Depth=1, Width=32


...

Running: Act=leaky_relu, Depth=3, Width=16


  warnings.warn(


Running: Act=leaky_relu, Depth=3, Width=32


  warnings.warn(


Running: Act=elu, Depth=1, Width=8
Running: Act=elu, Depth=1, Width=16
Running: Act=elu, Depth=1, Width=32
Running: Act=elu, Depth=2, Width=8
Running: Act=elu, Depth=2, Width=16
Running: Act=elu, Depth=2, Width=32
Running: Act=elu, Depth=3, Width=8
Running: Act=elu, Depth=3, Width=16
Running: Act=elu, Depth=3, Width=32
Running: Act=selu, Depth=1, Width=8
Running: Act=selu, Depth=1, Width=16
Running: Act=selu, Depth=1, Width=32
Running: Act=selu, Depth=2, Width=8
Running: Act=selu, Depth=2, Width=16
Running: Act=selu, Depth=2, Width=32
Running: Act=selu, Depth=3, Width=8
Running: Act=selu, Depth=3, Width=16
Running: Act=selu, Depth=3, Width=32
Running: Act=gelu, Depth=1, Width=8
Running: Act=gelu, Depth=1, Width=16
Running: Act=gelu, Depth=1, Width=32
Running: Act=gelu, Depth=2, Width=8
Running: Act=gelu, Depth=2, Width=16
Running: Act=gelu, Depth=2, Width=32
Running: Act=gelu, Depth=3, Width=8
Running: Act=gelu, Depth=3, Width=16
Running: Act=gelu, Depth=3, Width=32
Running: Act=swish,

In [8]:
# Assuming results_df is generated from the previous code block
results_df.to_csv('feedforward_benchmark_results.csv', index=False)
print("Benchmark results saved to 'feedforward_benchmark_results.csv'")

Benchmark results saved to 'feedforward_benchmark_results.csv'


In [9]:
# Load the generated results CSV (if running separately)
# results_df = pd.read_csv('feedforward_benchmark_results.csv')

# --- Generate Heatmaps ---
for activation in results_df['activation'].unique():
    # Filter data for the current activation
    df_act = results_df[results_df['activation'] == activation]
    
    # Pivot the data: Depth (index) vs. Width (columns) with val_mse (values)
    heatmap_data = df_act.pivot(index='depth', columns='width', values='val_mse')
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(
        heatmap_data, 
        annot=True, 
        fmt=".3f", 
        cmap="coolwarm_r", # '_r' reverses the colormap, so better (lower) MSE is darker
        cbar_kws={'label': 'Best Validation MSE'}
    )
    #[cite_start]
    plt.title(f'Validation MSE Benchmark: {activation.upper()}')
    plt.xlabel('Width (Neurons per Layer)')
    plt.ylabel('Depth (Hidden Layers)')
    plt.tight_layout()
    plt.savefig(f'heatmap_{activation}_val_mse.png')
    plt.close()

print("Heatmaps generated (one per activation function).")

Heatmaps generated (one per activation function).


In [10]:
# --- Find Best Model per Activation for R2 comparison ---
# Group by activation and find the row with the maximum test_r2 in each group
best_r2_per_activation = results_df.loc[results_df.groupby('activation')['test_r2'].idxmax()]

# --- Generate R2 Bar Chart ---
plt.figure(figsize=(10, 6))
sns.barplot(
    x='activation', 
    y='test_r2', 
    data=best_r2_per_activation.sort_values(by='test_r2', ascending=False)
)
plt.title('Best Test R-Squared ($R^2$) by Activation Function')
plt.xlabel('Activation Function')
plt.ylabel('Test $R^2$ (Higher is Better)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('bar_chart_best_r2_by_activation.png')
plt.close()

print("R-Squared Bar Chart generated.")

R-Squared Bar Chart generated.


In [11]:
# --- Track Overall Top 3 Models ---
model_summary = {
    'name': f'{activation}_D{depth}_W{width}',
    'test_r2': test_r2,
    'history': history.history  # <--- THIS IS THE CRITICAL DATA
}

# Keep the top 3 models based on R2 (higher is better)
best_overall_models.append(model_summary)
best_overall_models.sort(key=lambda x: x['test_r2'], reverse=True)
best_overall_models = best_overall_models[:3]

In [12]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming the 'best_overall_models' list still exists in your notebook's memory
# from when you ran the main execution loop.

# --- Function to plot loss curves ---
def plot_loss_history(history_dict, model_name, plot_filename):
    """Plots training and validation loss curves for a single model."""
    epochs = range(1, len(history_dict['loss']) + 1)
    
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, history_dict['loss'], 'b-', label='Training Loss (MSE)')
    plt.plot(epochs, history_dict['val_loss'], 'r-', label='Validation Loss (MSE)')
    
    # Highlight the best epoch (where validation loss was lowest)
    best_val_loss = min(history_dict['val_loss'])
    best_epoch = np.argmin(history_dict['val_loss']) + 1
    
    plt.axvline(best_epoch, color='k', linestyle='--', alpha=0.5, label=f'Early Stop Epoch ({best_epoch})')

    plt.title(f'Loss Convergence: {model_name} (Best Val MSE: {best_val_loss:.2f})')
    plt.xlabel('Epochs')
    plt.ylabel('Loss (Mean Squared Error)')
    plt.legend()
    plt.grid(True, which="both", linestyle='--', linewidth=0.5)
    plt.savefig(plot_filename)
    plt.close()

print("Generating Loss Curves for Top 3 Models...")
    
# --- Iterate and Plot Top 3 Models ---
for i, model_summary in enumerate(best_overall_models):
    model_name = model_summary['name']
    history_data = model_summary['history']
    filename = f'loss_curve_top_{i+1}_{model_name}.png'
    
    plot_loss_history(history_data, model_name, filename)
    print(f"Generated: {filename}")

print("\nAll Top-3 Loss Curves generated successfully.")

Generating Loss Curves for Top 3 Models...
Generated: loss_curve_top_1_leaky_relu_D3_W32.png
Generated: loss_curve_top_2_relu_D3_W32.png
Generated: loss_curve_top_3_gelu_D3_W32.png

All Top-3 Loss Curves generated successfully.
